In [1]:
import datetime
import sys
import os

import IPython
import IPython.display
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from sklearn.preprocessing import MinMaxScaler

import src.preprocessing_3days
from src.preprocessing_3days import series_to_supervised, preprocess
from src.functions import load_data, TimeSeriesTensor, create_evaluation_df, plot_train_history, validation, save_model, load_model

np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_seq_items = 2000

In [2]:
def train_test_split(df, n_test):
    if len(df) < 8760:
        n_test = round(len(df) * 0.2)
    test_df = df.copy()[-(n_test+71):]
    train_df = df.copy()[:-(len(test_df)-71)]
    return train_df, test_df

In [7]:
def MIMO_fulldata_preparation(df, n_test=4380, T=72, HORIZON=72):
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    df = preprocess(df, 'Belgium')
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    print(train_df.columns)
    #tensor_structure = {'X':(range(-T+1, 1), train_df.columns[:1]), 'X2':(range(1, 73), train_df.columns[1:6]), 'static':(None, train_df.columns[6:])}
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns[:1]), 'X2':(range(1, 73), train_df.columns[1:])}
    #tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    #print(tensor_structure[0])
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [8]:
train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(GEP4, n_test=4380, T=72, HORIZON=72)

Index(['value', 'value(t-168)', 'fractional hour_sin', 'fractional hour_cos',
       'day of year_sin', 'day of year_cos', 'working day', 'week_1', 'week_2',
       'week_3', 'week_4', 'week_5', 'week_6', 'week_7'],
      dtype='object')


In [4]:
GEP1 = pd.read_csv('../data/GEP/Consumption_1H.csv', index_col=0, header=0, names=['value'])
GEP4 = pd.read_csv('../data/GEP/B4_Consumption_1H.csv', index_col=0, header=0, names=['value'])
datasets = [GEP1, GEP4]
names = ['GEP1', 'GEP4']

In [19]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 72


FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(HORIZON, 14)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])


metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dT_train = []
dX_test = []
dX_scaler = []
for i,df in enumerate(datasets):
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df, n_test=4380, T=72, HORIZON=72)
    dX_train.append(tf.concat([train_inputs['X'],train_inputs['X2']], axis=2))
    dT_train.append(train_inputs['target'])
    dX_test.append(test_inputs)
    dX_scaler.append(y_scaler)
global_inputs_X = tf.concat(dX_train, 0)
global_inputs_T = tf.concat(dT_train, 0)
#test_inputs = pd.concat(dn_test, axis=1)

# full data LSTM MIMO compilation and fit
FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
        
history = FULL_LSTMIMO.fit(global_inputs_X, global_inputs_T, batch_size=1500, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)
save_model(FULL_LSTMIMO, 'new_inputs_model')
for i,df in enumerate(datasets):
    concat_input = tf.concat([dX_test[i]['X'],dX_test[i]['X2']], axis=2)
    FD_predictions = FULL_LSTMIMO.predict(concat_input)
    FD_eval_df = create_evaluation_df(FD_predictions, dX_test[i], HORIZON, dX_scaler[i])
    mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
    mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
    rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
    metrics.loc[i] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/global/3days/revised2_LSTM.csv')

Train on 43392 samples, validate on 7658 samples
Epoch 1/100
43392/43392 [==============================] - 24s 550us/sample - loss: 0.1012 - mean_squared_error: 0.1012 - val_loss: 0.0670 - val_mean_squared_error: 0.0670
Epoch 2/100
43392/43392 [==============================] - 20s 452us/sample - loss: 0.0467 - mean_squared_error: 0.0467 - val_loss: 0.0441 - val_mean_squared_error: 0.0441
Epoch 3/100
43392/43392 [==============================] - 17s 386us/sample - loss: 0.0368 - mean_squared_error: 0.0368 - val_loss: 0.0416 - val_mean_squared_error: 0.0416
Epoch 4/100
43392/43392 [==============================] - 17s 384us/sample - loss: 0.0349 - mean_squared_error: 0.0349 - val_loss: 0.0395 - val_mean_squared_error: 0.0395
Epoch 5/100
43392/43392 [==============================] - 17s 383us/sample - loss: 0.0327 - mean_squared_error: 0.0327 - val_loss: 0.0364 - val_mean_squared_error: 0.0364
Epoch 6/100
43392/43392 [==============================] - 18s 411us/sample - loss: 0.0293 

43392/43392 [==============================] - 16s 380us/sample - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.0197 - val_mean_squared_error: 0.0197
Epoch 96/100
43392/43392 [==============================] - 16s 379us/sample - loss: 0.0126 - mean_squared_error: 0.0126 - val_loss: 0.0196 - val_mean_squared_error: 0.0196
Epoch 97/100
43392/43392 [==============================] - 17s 381us/sample - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.0193 - val_mean_squared_error: 0.0193
Epoch 98/100
43392/43392 [==============================] - 16s 378us/sample - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.0196 - val_mean_squared_error: 0.0196
Epoch 99/100
43392/43392 [==============================] - 16s 380us/sample - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.0194 - val_mean_squared_error: 0.0194
Epoch 100/100
43392/43392 [==============================] - 17s 383us/sample - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.0193 - val_m

In [16]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 72
n_cell_lstm = 32

# Add static input via functional api
dynamic_input = Input(shape=[HORIZON, 6], name='time-series')
static_input = Input(shape=[22], name='one-hot')

static_out = (static_input)
x = tf.keras.layers.LSTM(n_cell_lstm, return_sequences=True)(dynamic_input)
x = tf.keras.layers.Flatten()(x)
dynamic_out = (x)
z = tf.keras.layers.concatenate([dynamic_out, static_out])
z = tf.keras.layers.Dense(HORIZON, activation='relu')(z)
main_output = tf.keras.layers.Dense(HORIZON, activation='softmax', name='main_output')(z)

FULL_LSTMIMO = Model(inputs=[dynamic_input, static_input], outputs=[z])


metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dstatic_train = []
dT_train = []
dX_test = []
dstatic_test = []
dX_scaler = []
for i,df in enumerate(datasets):
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df, n_test=4380, T=72, HORIZON=72)
    concat_input = tf.concat([train_inputs['X'],train_inputs['X2']], axis=2)
    dX_train.append(concat_input)
    dstatic_train.append(train_inputs['static'])
    dT_train.append(train_inputs['target'])
    dX_test.append(test_inputs)
    dstatic_test.append(test_inputs['static'])
    dX_scaler.append(y_scaler)
global_inputs_X = tf.concat(dX_train, 0)
global_inputs_static = tf.concat(dstatic_train, 0)
global_inputs_T = tf.concat(dT_train, 0)
#test_inputs = pd.concat(dn_test, axis=1)

# full data LSTM MIMO compilation and fit
FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
        
history = FULL_LSTMIMO.fit([global_inputs_X, global_inputs_static], global_inputs_T, batch_size=1500, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)
save_model(FULL_LSTMIMO, 'new_inputs_model')
for i,df in enumerate(datasets):
    concat_input = tf.concat([dX_test[i]['X'],dX_test[i]['X2']], axis=2)
    FD_predictions = FULL_LSTMIMO.predict([concat_input, dstatic_test[i]])
    FD_eval_df = create_evaluation_df(FD_predictions, dX_test[i], HORIZON, dX_scaler[i])
    mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
    mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
    rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
    metrics.loc[i] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/global/3days/static_LSTM.csv')

Train on 43392 samples, validate on 7658 samples
Epoch 1/100
 1500/43392 [>.............................] - ETA: 24sWARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: 


KeyboardInterrupt: 